In [1]:
#This part is to get data from Google BigQuery

%load_ext google.cloud.bigquery

In [2]:
import pandas as pd
from google.cloud import bigquery

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './Labor Statistics Project-27f747351078.json'

In [4]:
%%bigquery sample_data
select 
value,predicted_value,date
from `labor-statistics-project.export_evaluated_examples_bls_sample_data_20190803020030_2019_08_03T13_12_34_235Z.evaluated_examples`

In [5]:
sample_data

,value,predicted_value,date
0,140.239,[{'tables': {'prediction_interval': {'end': 0....,2018-01-01 00:00:00+00:00
1,140.805,[{'tables': {'prediction_interval': {'end': 0....,2018-02-01 00:00:00+00:00
2,141.142,[{'tables': {'prediction_interval': {'end': 0....,2018-03-01 00:00:00+00:00
3,141.662,[{'tables': {'prediction_interval': {'end': 0....,2018-04-01 00:00:00+00:00
4,142.146,[{'tables': {'prediction_interval': {'end': 0....,2018-05-01 00:00:00+00:00
...,...,...,...
691,101.678,[{'tables': {'prediction_interval': {'end': 0....,2018-08-01 00:00:00+00:00
692,101.376,[{'tables': {'prediction_interval': {'end': 0....,2018-09-01 00:00:00+00:00
693,101.284,[{'tables': {'prediction_interval': {'end': 0....,2018-10-01 00:00:00+00:00
694,101.447,[{'tables': {'prediction_interval': {'end': 0....,2018-11-01 00:00:00+00:00


In [6]:
#Get Row Count of the Data Frame
row_count=sample_data.date.count()

In [7]:
#Review the predicted value JSON format
sample_data.predicted_value[1]

[{'tables': {'prediction_interval': {'end': 0.0, 'start': 0.0},
   'value': 137.99078369140625}}]

In [8]:
#Function to extract the predicted value from the JSON format
def between(value, a, b):
    # Find and validate before-part.
    pos_a = value.find(a)
    if pos_a == -1: return ""
    # Find and validate after part.
    pos_b = value.rfind(b)
    if pos_b == -1: return ""
    # Return middle part.
    adjusted_pos_a = pos_a + len(a)
    if adjusted_pos_a >= pos_b: return ""
    return value[adjusted_pos_a:pos_b]

In [23]:
#To make sure that the function works
float(between(str(sample_data.predicted_value[1]),"value': ", "}}"))

137.99078369140625

In [24]:
#Define a Dataframe 
df=pd.DataFrame(columns=['Date','Value','Predict_Value'],index=range(0,row_count))

In [28]:
#Load the Date, Actual Value and the Predicted Value into the new dataframe df
for i in range(row_count):
    pv=str(sample_data.predicted_value[i])
    df.Date[i]=sample_data.date[i].strftime('%Y-%m-%d')
    df.Value[i]=sample_data.value[i]
    df.Predict_Value[i]=float(between(pv,"value': ", "}}"))
    

In [29]:
#Review Dataframe
df

,Date,Value,Predict_Value
0,2018-01-01,140.239,138.292
1,2018-02-01,140.805,137.991
2,2018-03-01,141.142,138.224
3,2018-04-01,141.662,138.606
4,2018-05-01,142.146,138.225
...,...,...,...
691,2018-08-01,101.678,104.055
692,2018-09-01,101.376,103.965
693,2018-10-01,101.284,104.231
694,2018-11-01,101.447,103.965


In [30]:
type(df.Predict_Value[1])

float

In [19]:
df.groupby(['Date'], as_index=False)['Value','Predict_Value'].sum()


,Date,Value,Predict_Value
0,2017-07-01,4009.227,138.3385925292968892.49623107910156116.8097152...
1,2017-08-01,4016.194,138.2615661621093893.44759368896484116.8499603...
2,2017-09-01,4041.757,138.4502258300781295.27804565429688116.9440765...
3,2017-10-01,4033.337,138.4530334472656296.6732177734375116.91426849...
4,2017-11-01,4032.456,138.2203063964843895.25031280517578116.7219696...
5,2017-12-01,4023.389,137.862380981445393.39073181152344115.95623779...
6,2018-01-01,4044.166,138.2923431396484492.88462829589844116.3352508...
7,2018-02-01,4063.590,137.9907836914062594.1978759765625116.16152954...
8,2018-03-01,4069.333,138.224487304687595.42582702636719116.63780975...
9,2018-04-01,4086.249,138.6063385009765695.7724609375117.10721588134...


In [13]:
#Prepare for creating the visualization through flask & chartjs
from flask import Flask, Markup, render_template

app = Flask(__name__)

In [14]:
colors = [
    "#F7464A", "#46BFBD", "#FDB45C", "#FEDCBA",
    "#ABCDEF", "#DDDDDD", "#ABCABC", "#4169E1",
    "#C71585", "#FF4500", "#FEDCBA", "#46BFBD"]

In [ ]:
df =df.sort_values(by=['Date'])

In [15]:
labels = df.Date
value1 = df.Value
value2 = df.Predict_Value

In [18]:
@app.route('/test')
def test5():
        line_labels=labels
        line_values=values
        return render_template('test_chart.html', title='Test_Chart', max=200, labels=line_labels, values=line_values)

In [136]:
@app.route('/')
def hello():
        return render_template('hello.html', title="Weclome to Leo Song's Project Page")


In [137]:
if __name__ == '__main__':
    app.run(host='localhost', port=8081)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8081/ (Press CTRL+C to quit)
[2019-08-28 21:42:13,567] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\users\leveo\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\leveo\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\leveo\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\leveo\appdata\local\programs\python\python37\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\leveo\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\leveo\appdata\local\programs